In [1]:
import pandas as pd
from io import open
import json as json
import re
import numpy as np
import pickle


In [2]:
archivo = open("../osm/edificio_este.osm","r")

osm = archivo.readlines()

In [3]:
def save_dict(dictionary,file):
    archivo = open(file, "w")
    json.dump(dictionary,archivo,indent=2)
    print("saved:",file)
    archivo.close()

def read_osm(file):
    tmp = open(file)
    osm = tmp.readlines()
    return osm

def get_dictionary(osm,OSM_OBJETO,kind="dict"):
    # identifica donde inicia un objeto y donde 
    # termina, y los pone en una lista    
    objeto = []
    coma   = []
    for i,renglon in enumerate(osm):
        if renglon.startswith("OS:"): #localiza el inicio de un objeto
            objeto.append(i)
        if ";" in renglon:  #localiza el final de un objeto
            coma.append(i)
#     en os_lista, guarda donde inicia y termina cada objeto
    osm_lista = []
    for i,j in zip(objeto,coma):
        osm_lista.append(osm[i:j])
# separa en dos listas, lista objeto y resto del osm
    objeto_lista = []
    resto_lista  = []
    for objeto in osm_lista:
        if OSM_OBJETO in objeto:
            objeto_lista.append(objeto)
        else:
            resto_lista.append(objeto)
#     print(objeto_lista)
#             Limpia la lista que contiene el objeto
    lista_objetos = []
    for objeto in objeto_lista:
        lista_objeto = []
        objeto[0]= objeto[0].replace(",","")
        for campo in objeto:
            lista_objeto.append(campo.replace("\n","").
                                 replace("","").
#                                  replace("{","").
#                                  replace("}","").
                                 replace("!- ","").
                                 strip())
        lista_objetos.append(lista_objeto)
    
    diccionario = {}
    for objeto in lista_objetos:
        nombre_objeto,_ = objeto[2].split(",")
        tmp = {}
        for propiedad in range(1,len(objeto)):
            try:
                valor,nombre = objeto[propiedad].split(",")
                try:
                    valor = float(valor)
                except:
                    if nombre=="Thermal Absorptance":
                        valor = 0.9
                    if nombre=="Solar Absorptance":
                        valor = 0.7
                    if nombre=="Visible Absorptance":
                        valor = 0.7
                    if nombre=="Roughness":
                        valor = "Smooth"
                    if nombre=="Thickness m":
                        valor = 0.1
                    if nombre=="Conductivity W/m-K": 
                        valor = 0.1
                    if nombre=="Density kg/m3": 
                        valor = 0.1
                    if nombre=="Specific Heat J/kg-K": 
                        valor = 0.1
            except:
                pass
            tmp.update({nombre:valor})
        del tmp["Name"]
        diccionario.update({nombre_objeto:tmp})
# diccionario
    if kind=="dict":
        return diccionario
    if kind=="list":
        return lista_objetos
osm = read_osm("../osm/edificio_este.osm")
diccionario = get_dictionary(osm,"OS:Material,\n")

In [4]:
osm = read_osm("../osm/edificio_este.osm")
diccionario = get_dictionary(osm,"OS:Material,\n")


In [5]:
for objeto in diccionario:
    print(objeto)
diccionario['mortero1:4_L1_a0pX']

concreto_fc150_granzon_L8_a0pX
concreto_fc150_L6_a0pX
concreto_fc250_L30_a0pX
madera_L2p2_a0pX
marmol_L2_a0pX
concreto_fc250_L20_a0pX
mortero1:4_L1_a0pX
loseta_Lx_a0pX
impermeabilizante_LX_a0p3
fachaleta_L1_a0p3
concreto_fc200_L15_a0pX
durock_L1p27_a0p3
concreto_fc250_L15_a0pX
concreto_fc150_marmol_L8_a0pX
concreto_fc150_L8_a0pX
crest_L1_a0pX
tabique_hueco_L1_a0pX
concreto_fc150_L5_a0pX
lana_L6p35_a0p3
tabique_hueco_L2_a0pX
tabique_hueco_L3_a0pX
vermiculita_LX_a0pX
concreto_fc200_L20_a0pX
aluminio
Metal Decking
fachaleta_L1_a0p7
F16 Acoustic tile


{'Handle': '{da0ae925-c712-45dc-a93f-babc664e7911}',
 'Roughness': 'Smooth',
 'Thickness {m}': 0.01,
 'Conductivity {W/m-K}': 1.0,
 'Density {kg/m3}': 1800.0}

In [6]:
diccionario["concreto_fc150_granzon_L8_a0pX"]["Solar Absorptance"] = 0.2

In [7]:
diccionario["concreto_fc150_granzon_L8_a0pX"]["Solar Absorptance"] 

0.2

In [8]:
save_dict(diccionario,"dictionario.json")

saved: dictionario.json


In [9]:
def load_dict(file):
    with open(file) as json_file:
        dictionary = json.load(json_file)
    return dictionary
nuevo = load_dict("dictionario.json")

In [10]:

def save_osm(osm,OSM_OBJETO):
    # identifica donde inicia un objeto y donde 
    # termina, y los pone en una lista    
    objeto = []
    coma   = []
    for i,renglon in enumerate(osm):
        if renglon.startswith("OS:"): #localiza el inicio de un objeto
            objeto.append(i)
        if ";" in renglon:  #localiza el final de un objeto
            coma.append(i)
#     en os_lista, guarda donde inicia y termina cada objeto
    osm_lista = []
    for i,j in zip(objeto,coma):
        osm_lista.append(osm[i:j])
# separa en dos listas, lista objeto y resto del osm
    objeto_lista = []
    resto_lista  = []
    for objeto in osm_lista:
        if OSM_OBJETO in objeto:
            objeto_lista.append(objeto)
        else:
            resto_lista.append(objeto)
    return objeto_lista, resto_lista

In [11]:
parte_materiales, resto = save_osm(osm,"OS:Material,\n")

In [12]:
# def save
# with open("test.osm", "w") as f:
#     for s in osm:
#         f.write(s)

SyntaxError: invalid syntax (<ipython-input-12-5bbd9a6405e9>, line 1)

In [ ]:
OS:Material,
  {da0ae925-c712-45dc-a93f-babc664e7911},!- Handle
  mortero1:4_L1_a0pX,!- Name
  MediumRough,!- Roughness
  0.01,!- Thickness {m}
  1.0,!- Conductivity {W/m-K}
  1800.0,!- Density {kg/m3}
  1000.0;!- Specific Heat {J/kg-K}

OS:Material,
  {d2d407be-ecda-4a03-9396-eac43147510b},!- Handle
  impermeabilizante_LX_a0p3,!- Name
  VeryRough,!- Roughness
  0.0095,!- Thickness {m}
  0.16,!- Conductivity {W/m-K}
  1121.29,!- Density {kg/m3}
  1460.0,!- Specific Heat {J/kg-K}
  ,!- Thermal Absorptance
  0.7;!- Solar Absorptance

In [13]:
objeto_archivo = []

for objeto in diccionario:
    print("OS:Material,")
    print(diccionario[objeto]["Handle"],",!- Handle")    
    print(objeto,",!- Name")
    print(diccionario[objeto]["Roughness"],",!- Roughness")  
    print(diccionario[objeto]["Thickness {m}"],",!- Thickness {m}")  
    print(diccionario[objeto]["Density {kg/m3}"],",!- Density {kg/m3}")  
    print(diccionario[objeto]["Specific Heat {J/kg-K}"],",!- Specific Heat {J/kg-K}")  
    print("\n")


OS:Material,
{2e7157dc-32c2-44e3-9415-7874ececbba9} ,!- Handle
concreto_fc150_granzon_L8_a0pX ,!- Name
Smooth ,!- Roughness
0.15 ,!- Thickness {m}
1800.0 ,!- Density {kg/m3}
1000.0 ,!- Specific Heat {J/kg-K}


OS:Material,
{acc9b317-088e-4fa3-a849-9aec59e11271} ,!- Handle
concreto_fc150_L6_a0pX ,!- Name
Smooth ,!- Roughness
0.15 ,!- Thickness {m}
1800.0 ,!- Density {kg/m3}
1000.0 ,!- Specific Heat {J/kg-K}


OS:Material,
{fe21d530-02b9-4f7e-8369-6955e97efd5a} ,!- Handle
concreto_fc250_L30_a0pX ,!- Name
Smooth ,!- Roughness
0.3 ,!- Thickness {m}
1800.0 ,!- Density {kg/m3}
1000.0 ,!- Specific Heat {J/kg-K}


OS:Material,
{8f09ab24-4c76-41a0-aafb-911a9de403f8} ,!- Handle
madera_L2p2_a0pX ,!- Name
Smooth ,!- Roughness
0.02 ,!- Thickness {m}
600.0 ,!- Density {kg/m3}
1210.0 ,!- Specific Heat {J/kg-K}


OS:Material,
{9c0ea449-48e9-4c01-b8c0-5a414bb4b01c} ,!- Handle
marmol_L2_a0pX ,!- Name
Smooth ,!- Roughness
0.02 ,!- Thickness {m}
2700.0 ,!- Density {kg/m3}
879.0 ,!- Specific Heat {J/kg-K}


KeyError: 'Specific Heat {J/kg-K}'

In [ ]:
diccionario